In [1]:
# Home assignment from Wirecard.
# received 16.01 
# Notebook for Task 3 and 4

# Task 3 and 4

The aim of this assignment is to fit a model to the data, so that predictions about the customers’ purchase behavior in future can be made. The model is defined by four parameters namely r, α, a, b and they are always greater than 0. For fitting the model (i.e. finding the parameters r, α, a, b), Maximum Likelihood Estimation (MLE) can be used. The log likelihood (LL) function which is to be maximized is given as follows:

![title](img/NLL.png)

Implement the objective function given by Eq. 2 as an optimizable (minimizable) function. Note that the natural logarithm of the Gamma function is already available in different programming languages, so this can be used directly. Details about the function are:

* Input parameters: x, tx, T for all customers
* Model parameters: r, α, a, b
* Output parameter: The value of the function (NLL i.e. Eq. 2) evaluated at some value of r, α, a, b given the customers' data x, tx and T.

Minimize this function using a numerical method for non-linear optimization problems e.g. the Nelder-Mead algorithm (which does not require calculation of derivatives). Alternately, other algorithms e.g. L-BFGS can be used, gradient calculations can be performed using numerical approximations.

The starting point for parameters r, α, a, b can be taken as the result of evaluating Gaussian functions with mean 1.0 and standard deviation 0.05. Also, these four parameters are always greater than 0, so in case during optimization, if any of these parameters tend to go towards negative, this can be handled for example by returning infinity as the output parameter of the objective function.

The parameters tx and T have to be normalized before passing them to the function. For this, do the following:
- Get the maximum value of T from the available values
- Calculate the normalization factor by dividing 10.0 by the maximum value of T
- Multiply every element in tx and T vector by this factor

Also, the model parameter α has the same units like tx and T, so it also has to be multiplied by the normalization factor inside the objective function before any evaluations. To summarize, while calculating Eq. 2, use scaled α, tx and T values.

Find the optimal values of r, α, a, b that minimize the objective function and define a strategy to check convergence from different starting points. Save these parameter estimates in a csv file called estimated_parameters.csv.

In [2]:
#Import packages
import pandas as pd
import numpy as np
import scipy as sp
# treatment of the warnings
import warnings
# ln of the Gamma function 
from scipy.special import gammaln as lnG
# for minimization
from scipy import optimize
# write the final parameters
import csv

In [3]:
# load the .csv file
df = pd.read_csv('csv/summary_customers.csv',index_col='Unnamed: 0')
df.head()

,Customer ID,x,tx,T
0,1,2,30.43,38.86
1,2,1,1.71,38.86
2,3,0,0.00,38.86
3,4,0,0.00,38.86
4,5,0,0.00,38.86


## CustomFunction class

Implementation of a custom optimizable (minimizable) function 

In [4]:
class CustomFunction:
    def __init__(self, verbose=False):
        """Constructor + initialization of the fit parameters"""
        self.verbose = verbose
        #initialize model parameters
        self.__initialize_par()
        
    def __obj_f(self, params, df):
        """Implementation of the objective function.
        
        NLL = -1/N Sum( lnA1i + lnA2i + ln( exp(lnA3i) + delta exp(lnA4i)) )
        """
        #update parameters
        self.__update_params(params)
        
        #normalise alpha
        alpha_norm =self.alpha * self.norm
        # protection from negative parameters
        if self.r <=0 or self.b <=0 or self.a <= 0 or alpha_norm <= 0:
            return np.inf
        # number os samples
        N = len(df)
        # loss function
        with warnings.catch_warnings():
            # catch warnings from "bad" values in the log. 
            # they are treated with the corresponding value of the NLL
            warnings.simplefilter("ignore",RuntimeWarning)
            obj_i = self.__loss(df,self.r,alpha_norm,self.a,self.b)
        # protection from the inf values that might arise from division by zero
        if obj_i.isin([np.inf]).values.any():
            return np.inf
        # normalised NLL
        out = obj_i.sum() * (-1) / N
        return out
    
    def __loss(self, df,r,alpha,a,b):
        """Implementation of the loss function.
        """
        lnA1 = lnG(r+df.x) + r*np.log(alpha) - lnG(r)
        lnA2 = lnG(a+b) + lnG(b+df.x) - lnG(b) - lnG(a+b+df.x)
        lnA3 = -(r+df.x)*np.log(alpha+df['T'])
        lnA4 = np.log(a) - np.log(b+df.x-1) - (r+df.x)*np.log(alpha + df.tx)
        deltaA4 = self.__deltaA4(df.x,np.exp(lnA4))
        out = lnA1 + lnA2 + np.log(np.exp(lnA3) + deltaA4)
        return out
    
    def __deltaA4(self,x,A4):
        """Implementation of the delta function.
        deltaX = 0 if x <= 0 
        deltaX = X otherwise
        """
        A4[x <= 0] = 0
        return A4
    
    def __update_params(self, new_params):
        """Update model parameters
        """
        self.r = new_params[0].copy()
        self.alpha = new_params[1].copy()
        self.a = new_params[2].copy()
        self.b = new_params[3].copy()
    
    def __initialize_par(self):
        """Method to initialize parameters with Gaus(1,0.05)
        """
        self.r, self.alpha, self.a, self.b = np.random.normal(1, 0.05,4)
    
    def set_r(self, r):
        """Method to set parameter r
        """        
        if r <= 0: 
            raise ValueError('r should be positive')
        self.r = r
        
    def set_alpha(self, alpha):
        """Method to set parameter alpha
        """
        if alpha <= 0: 
            raise ValueError('alpha should be positive')
        self.alpha = alpha
    
    def set_a(self, a):
        """Method to set parameter a
        """        
        if a <= 0: 
            raise ValueError('a should be positive')
        self.a = a
        
    def set_b(self, b):
        """Method to set parameter b
        """
        if b <= 0: 
            raise ValueError('b should be positive')
        self.b = b
    
    def set_parameters(self, r, alpha, a, b):
        """Method to set the fit parameters
        """
        self.set_r(r), self.set_alpha(alpha), self.set_a(a), self.set_b(b)
    
    def fit(self, df, minimizer_strategy = [(2,'Nelder-Mead')]):
        """Implementation of the custom minimisation procedure.
        
        Parameters:
        df - pandas DataFrame with the input data of the next format (x,tx,T),
        minimizer_strategy - list of pears specifying the minimzersand number of calls to be used
        (default [(2,'Nelder-Mead')])
        """
        # check the input data
        self.__validate_df(df)
        # compute the scalefactor and normalise the data
        norm_df = self.__normalise_data(df)
        # initial parameters
        initial_pars = [self.r,self.alpha,self.a,self.b]
    
        for num_iter,minimizer in minimizer_strategy:
            print('Run {} minimizer'.format(minimizer))
            for i in range(1,num_iter+1):
                print('Step ',i)
                initial_pars = [self.r,self.alpha,self.a,self.b]
                if self.verbose: print('Initialisation: r = ',self.r, ' alpha = ',self.alpha, ' a = ',self.a, ' b = ',self.b)
                # perform the minimization
                self.fit_res = sp.optimize.minimize(self.__obj_f,x0 = initial_pars,args=(norm_df), method=minimizer) 
                # update the model parameters
                self.__update_params(self.fit_res.x)
                if i!= num_iter+1 and self.verbose:
                    self.print_final_results()
                    print('\n')
                
    def fit_global(self, df, minimizer_strategy = 'Nelder-Mead',num_iter = 100):
        """Implementation of the global (brut-force) minima finding algorithm - basinhopping
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.basinhopping.html#scipy.optimize.basinhopping
        Large number of iterations is recommended due to a step-based optimisation procedure.
        Fit succeed if 15% of num_iter the global minimum candidate remains the same.
        
        Parameters:
        df - pandas DataFrame with the input data of the next format (x,tx,T),
        minimizer_strategy - The minimization method (default: Nelder-Mead),
        num_iter - number of max iterations (default: 100)
        """
        # check the input data
        self.__validate_df(df)
        # compute the scalefactor and normalise the data
        norm_df = self.__normalise_data(df)
        # initial parameters
        initial_pars = [self.r,self.alpha,self.a,self.b]
        print('********Basin-hopping minimisation********')
        if self.verbose: print('Initialisation: r = ',self.r, ' alpha = ',self.alpha, ' a = ',self.a, ' b = ',self.b)
        # perform the minimisation
        self.fit_res = sp.optimize.basinhopping(self.__obj_f,x0 = initial_pars, 
                                            minimizer_kwargs={"method": minimizer_strategy,'args':(norm_df)},
                                            stepsize = 0.05,
                                            niter = num_iter,
                                            niter_success = int(num_iter*0.15),
                                            disp = self.verbose)
        # update the model parameters
        self.__update_params(self.fit_res.x)
    
    def __validate_df(self,df):
        """Method to check whether the input DataFrame contains
        required Series (x,xt,T) and free of NaNs"""
        cols = ['x','tx','T']
        for c in cols:
            if c not in df.columns:
                raise ValueError('Column ' + c + ' is missing in the input DataFrame.')
    
    def __normalise_data(self,df):
        """Method to normalise parameters tx and T
        """
        df_temp = df.copy()
        self.__norm_sf(df)
        df_temp['tx']*=self.norm
        df_temp['T']*=self.norm
        return df_temp
    
    def __norm_sf(self,df):
        """Method to compute the normalisation factor
        """
        max_T = df['T'].max()
        self.norm = 10./ max_T
    
    def print_final_results(self):
        """Method to print final results
        """
        print('*********Minimisation is done*********')
        # For some minimzers success and status are not defined
        # e.g. doesn't work for basinhopping
        try:
            print('Success: ',self.fit_res.success)
            print('Status: ',self.fit_res.status)
        except Exception: pass
        print('NLL: ',self.fit_res.fun)
        print('Optimized parameters: r = ',self.r, ' alpha = ',self.alpha, ' a = ',self.a, ' b = ',self.b)
        if self.verbose:
            print(self.fit_res)
            
    def getFitResults(self):
        """Method to return the fit results
        """
        try:
            return self.fit_res
        except Exception:
            print('Fit has not been applied yet, no fit result object exist.')
            return None
    
    def getFitParameters(self,digits = 0):
        """Method to return the fit parameters
        """
        if digits != 0:
            return (round(x,digits) for x in self.getFitParameters())
        return self.r, self.alpha, self.a, self.b


## Minimizers

In [5]:
# Instantiate tyhe class object
model = CustomFunction(verbose=False)
# Fit with the custom minimization routine
model.fit(df,minimizer_strategy=[(2,'Nelder-Mead')])
# Fit with the global minimizer 
#model.fit_global(df,minimizer_strategy='Nelder-Mead',num_iter=300)
model.print_final_results()

Run Nelder-Mead minimizer
Step  1
Step  2
*********Minimisation is done*********
Success:  True
Status:  0
NLL:  2.6505481706776455
Optimized parameters: r =  0.24259123935904925  alpha =  4.413484054346327  a =  0.7928829005072499  b =  2.425763897154502


In [6]:
# write final parameters to the .csv file
with open('csv/estimated_parameters.csv','w') as file:
    writer = csv.writer(file,delimiter=',',)
    writer.writerow(['r','alpha','a','b'])
    writer.writerow(model.getFitParameters(2)) # rounded to 2 digits